构造量子神经网络，本线路来自于旦波老师的那篇文章，放在代码库的文献中

In [1]:
import qibo
from qibo import Circuit, gates
from qibo import hamiltonians
from qibo.symbols import X, Y, Z, I
from qibo.noise import NoiseModel, PauliError, DepolarizingError
from qibo.result import MeasurementOutcomes
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm, logm
from scipy.optimize import minimize
from scipy.special import comb
from itertools import combinations
import multiprocessing
from math import pi
qibo.set_backend("numpy")
from numpy.linalg import eigh

[Qibo 0.2.2|INFO|2024-01-10 17:23:35]: Using numpy backend on /CPU:0


In [2]:
# exp(sigma_I)
# 任意pauli string 的 exp 算符
def circ_exp_PS(circuit, pauli_string, theta):
    """exp[-i delta_beta a_mu simga^mu]"""
    for i in range(n_qubit): # 遍历所有qubit
        temp = pauli_string[i]
        if temp=='I':  # 前后加入门
            circuit.add(gates.Z(i))
        elif temp=='x':
            circuit.add(gates.H(i))
        elif temp=='y':
            circuit.add(gates.RX(q=i,theta=pi/2))

    for i in range(n_qubit-1): # 加入CNOT门
        circuit.add(gates.CNOT(i,n_qubit-1))
    circuit.add(gates.RZ(q=n_qubit-1,theta=theta))
    for i in range (n_qubit-1): # 第二次添加CNOT门
        temp = n_qubit-2 - i
        circuit.add(gates.CNOT(temp,n_qubit-1))
    
    for i in range(n_qubit): # 遍历所有qubit
        temp = pauli_string[i]
        if temp=='x':
            circuit.add(gates.H(i))
        elif temp=='y':
            circuit.add(gates.RX(q=i,theta=-pi/2))

# 线路U
def map_PS(string:str,position):
    """输入一个短的pauli string，和短pauli string的起始位置，补全剩下的单位算符和自动将超出n_qubit的字符替换掉最前面的单位算符，返回数据类型为str"""
    if len(string)<=n_qubit:
        temp = ''
        for i in range(n_qubit):
            temp += 'I'

        temp_list = list(temp)
        for i in range(len(string)):
            if position+i<=n_qubit-1:
                temp_list[position+i] = string[i]
            elif position+i > n_qubit-1:
                temp_list[position+i-n_qubit] = string[i]
        res = ''.join(temp_list)
        # print('pauli string:',res)
        return res
    else:
        print('pauli string lenth large than n_qubit')


# 表象变换矩阵
# Unitary Quantum Neural Network
def circ_U(circuit,alpha,layers):
    """U为表象变换矩阵，其中alpha为U参数（大小为3*n_qubuit），alpha的大小为3*n_qubit*layers"""
    for l in range(layers):
        for i in range(n_qubit): # 遍历所有qubit
            temp_PS = map_PS(string='x',position=i)
            circ_exp_PS(circuit=circuit, pauli_string=temp_PS, theta=alpha[3*l+i])
            temp_PS = map_PS(string='z',position=i)
            circ_exp_PS(circuit=circuit, pauli_string=temp_PS, theta=alpha[3*l+n_qubit+i])
            temp_PS = map_PS(string='zz',position=i)
            circ_exp_PS(circuit=circuit, pauli_string=temp_PS,theta=alpha[3*l+2*n_qubit+i])

In [4]:
n_qubit = 4
layers = 3
test_circ = Circuit(nqubits=n_qubit)
random_alpha = np.random.random(3*n_qubit*layers)
circ_U(circuit=test_circ,alpha=random_alpha,layers=layers)
print(test_circ.draw())

q0:     ─H─o────────────o─H─o────────────o───o────────────o─Z─o────────────o─Z ...
q1:     ─Z─|─o────────o─|─Z─|─o────────o─|───|─o────────o─|─H─|─o────────o─|─H ...
q2:     ─Z─|─|─o────o─|─|─Z─|─|─o────o─|─|─Z─|─|─o────o─|─|─Z─|─|─o────o─|─|─Z ...
q3:     ─Z─X─X─X─RZ─X─X─X─Z─X─X─X─RZ─X─X─X─Z─X─X─X─RZ─X─X─X─Z─X─X─X─RZ─X─X─X─Z ...

q0: ... ─o────────────o─Z─o────────────o─Z─o────────────o─Z─o────────────o─Z─o ...
q1: ... ─|─o────────o─|───|─o────────o─|─Z─|─o────────o─|─Z─|─o────────o─|─Z─| ...
q2: ... ─|─|─o────o─|─|───|─|─o────o─|─|─H─|─|─o────o─|─|─H─|─|─o────o─|─|───| ...
q3: ... ─X─X─X─RZ─X─X─X─Z─X─X─X─RZ─X─X─X─Z─X─X─X─RZ─X─X─X─Z─X─X─X─RZ─X─X─X───X ...

q0: ... ────────────o─Z─o────────────o─Z─o────────────o───o────────────o─H─o── ...
q1: ... ─o────────o─|─Z─|─o────────o─|─Z─|─o────────o─|─Z─|─o────────o─|─Z─|─o ...
q2: ... ─|─o────o─|─|─Z─|─|─o────o─|─|─Z─|─|─o────o─|─|─Z─|─|─o────o─|─|─Z─|─| ...
q3: ... ─X─X─RZ─X─X─X─H─X─X─X─RZ─X─X─X─H─X─X─X─RZ─X─X─X───X─X─X─RZ─X─X─X─Z─X─X ...

q